<a href="https://colab.research.google.com/github/shreya-prabhu/nlp/blob/master/2BertforAES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# **Automated Essay Evaluation**
---
#### **Phase** :
*   Coherence and Attention Mechanisms with Neural Networks and Pre Trained Bert (Semantic)


#GPU

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla K80


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.1MB/s 
     |████████████████████████████████| 901kB 40.1MB/s 
     |████████████████████████████████| 3.3MB 41.1MB/s 


In [ ]:
!pip install skll

     |████████████████████████████████| 133kB 4.1MB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
     |████████████████████████████████| 112kB 45.5MB/s 
     |████████████████████████████████| 552kB 31.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


## Dataset

### Truncate dataframe to use set#2

In [ ]:
!pip install --upgrade language_tool_python

# Import Pre-Trained Model

In [ ]:
from transformers import BertModel, BertTokenizer
# import BERT-base pretrained model
bert = BertModel.from_pretrained('bert-base-cased')
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
import torch.nn as nn
class BERT_Arch(torch.nn.Module):

    

    def __init__(self,freeze_bert=False):
        super(BERT_Arch, self).__init__()
        # bert=bert
        # bert = Bert.from_pretrained("bert-base-uncased")
        # bert.classifier=torch.nn.Linear(768,1)
        # bert.num_labels=1
        self.bert = bert
        
        self.pre_classifier = torch.nn.Linear(768, 512)
        # self.dropout = torch.nn.Dropout(0.07)
        self.classifier = torch.nn.Linear(512,1)
        # self.num_labels = 1

    def forward(self, input_ids, attention_mask):
        output_1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0, :]
        pooler = self.pre_classifier(pooler)
        # pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        # output = torch.nn.Sigmoid()(output)
        return output


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=3):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BERT_Arch(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/capstone_data/bert_set1.csv')
# df=df[10688:12257]
# df=df[12258:12979]
df=df[1785:3584]
df.dropna()
df.head(2)

,essay_set,essay,score
1785,2,Do you think that libraries should remove cert...,2.0
1786,2,"In @DATE1's world, there are many things found...",4.0


In [ ]:
print(df.score.unique())
print(df["score"].value_counts())

[2. 4. 5. 3. 1. 6.]
4.0    777
3.0    763
2.0    153
5.0     75
1.0     24
6.0      7
Name: score, dtype: int64


In [ ]:
import numpy as np
df.score=df.score.astype(np.float32)
print(df.shape)

(1799, 3)


# Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df.essay.values
y = df.score

X_train, X_val, y_train_in, y_val_in =\
    train_test_split(X, y, test_size=0.20,shuffle=False)

In [ ]:
y_train=y_train_in.values
y_val = y_val_in.values
print(len(y_train),len(y_val))

1439 360


## Preprocessing Essays 
#### encode_plus will:
*   (1) Tokenize the sentence
*   (2) Add the `[CLS]` and `[SEP]` token to the start and end
*   (3) Truncate/Pad sentence to max length
*   (4) Map tokens to their IDs
*   (5) Create attention mask
*   (6) Return a dictionary of outputs
  

In [ ]:
# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:

        encoded_sent = tokenizer.encode_plus(
            sent,  # Preprocess sentence
            None,
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN, 
            truncation=True,                 # Max length to truncate/pad
            padding='max_length',         # Pad sentence to max length
            return_token_type_ids=True,          # Return PyTorch tensor
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.as_tensor(input_ids)
    input_ids.float()
    attention_masks = torch.as_tensor(attention_masks)
    attention_masks.float()

    return input_ids, attention_masks

In [ ]:
MAX_LEN = 512

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
attention_mask = list(preprocessing_for_bert([X[0]])[1].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)
print('Attention Mask:',attention_mask)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Original:  Do you think that libraries should remove certain materials off the shelves? People have different oppions, of whats good and whats bad. I have read and seen a lot of books in my life time. I hear people telling me, 'oh dont read that book its a bad book.'But I ask myself, @CAPS2 do I know it's a bad book when I haven't even given it a chance?'      @CAPS1 are some books, music, movies, and magazines out @CAPS1 that are offensive. Yet we still want to read, listen, watch, and look at them. If we tried to remove all the offensive books, from the libraries we wouldn't have anything left on the shelves. Katherine Paterson said, 'If I have the right to remove that book from the shelf that work i abhor- then you also have exactly the same right and so does everyone else. And then we have no books left on the shelf for any of us.' Katherine Paterson makes a great point out of her quote. Why should we have to remove a book if just some people think its offensive?     Ask yourself t

# Create PyTorch DataLoader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.as_tensor(y_train)
train_labels.float()
val_labels = torch.tensor(y_val)
val_labels = torch.as_tensor(y_val)
val_labels.float()

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 6

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size,pin_memory=True,num_workers=2)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size,pin_memory=True,num_workers=2)

# Define Model Architecture for Regression



*   Tokenized representation of essay sent as input layer
*   Hidden Layer of Dimensions (768,512) 
*   Dropout applied on hidden layer
*   Output Layer of Dimensins (512,1)








The Linear Module computes output from input using a
linear function, and holds internal Tensors for its weight and bias.

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
import random
import time

# Specify loss function
# loss_fn = nn.NLLLoss(weight=weights)
# loss_fn=nn.BCEWithLogitsLoss(reduction='mean')
loss_fn = nn.MSELoss(reduction='mean')

# def set_seed(seed_value=42):
#     """Set seed for reproducibility.
#     """
#     random.seed(seed_value)
#     np.random.seed(seed_value)
#     torch.manual_seed(seed_value)
#     torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, epochs=3, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            # loss=[]
            batch_counts +=1
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad(set_to_none=True)

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)
            x=len(logits)
            logits=logits.view(x)


            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()


            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        # if evaluation == True:
        #     # After the completion of each training epoch, measure the model's performance
        #     # on our validation set.
        #     val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
        time_elapsed = time.time() - t0_epoch
            
        print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {time_elapsed:^9.2f}")
        print("-"*70)
        print("\n")
    
    print("Training complete!")



In [ ]:
# set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=3)
train(bert_classifier, train_dataloader, epochs=3, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   5.957279   |     -      |     -     |   8.38   
   1    |   40    |   0.812709   |     -      |     -     |   7.76   
   1    |   60    |   0.499724   |     -      |     -     |   7.80   
   1    |   80    |   0.554524   |     -      |     -     |   7.86   
   1    |   100   |   0.532418   |     -      |     -     |   7.88   
   1    |   120   |   0.508157   |     -      |     -     |   7.91   
   1    |   140   |   0.419290   |     -      |     -     |   7.93   
   1    |   160   |   0.414690   |     -      |     -     |   7.95   
   1    |   180   |   0.374609   |     -      |     -     |   7.98   
   1    |   200   |   0.448956   |     -      |     -     |   8.01   
   1    |   220   |   0.302667   |     -      |     -     |   8.04   
   1    |   239   |   0.432983   |     -      |     -     |   7.60   


In [ ]:
#Write predictions and essay row to file

In [ ]:

import torch.nn.functional as F
import numpy
def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    # all_logits = F.softmax(all_logits, dim=1).cpu().numpy()
    # probs= (all_logits).cpu.detatch.numpy()
    #print(all_logits)
    return all_logits.cpu().numpy()


In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader)

In [ ]:
#write to csv --- test set
df_to_csv = pd.DataFrame()
df_to_csv['essay_id'] = y_val_in.index
df_to_csv['essay'] = X_val
df_to_csv['score'] = y_val
df_to_csv['predicted_score']=probs


## Scale scores back up

In [ ]:
df_to_csv.head(20)

,essay_id,essay,score,predicted_score
0,3224,"To whom it @MONTH1 concern, Censorship is ...",5.0,4.220248
1,3225,It is a prominent question of debate whether o...,5.0,4.145442
2,3226,Children are our future because when I am old ...,4.0,3.607174
3,3227,"Books, music, movies, and magazines are all a ...",4.0,3.920864
4,3228,I think what ever is offensive should be taken...,2.0,2.787554
5,3229,I think they should be taken off the shelves i...,2.0,1.653491
6,3230,Katherine Paterson made a valid point in stati...,4.0,4.057228
7,3231,There are things in the world that are not men...,3.0,3.791177
8,3232,What do you go to the library for? In earlier...,3.0,3.926536
9,3233,A library card is one thing a kid always looke...,4.0,3.929719


In [ ]:
# df_to_csv['predicted_score']=df_to_csv['predicted_score'].round(1)

In [ ]:
# df_to_csv.head(20)

In [ ]:
df_to_csv['predicted_score']=df_to_csv['predicted_score'].round(0)

In [ ]:
df_to_csv.head(20)

,essay_id,essay,score,predicted_score
0,3224,"To whom it @MONTH1 concern, Censorship is ...",5.0,4.0
1,3225,It is a prominent question of debate whether o...,5.0,4.0
2,3226,Children are our future because when I am old ...,4.0,4.0
3,3227,"Books, music, movies, and magazines are all a ...",4.0,4.0
4,3228,I think what ever is offensive should be taken...,2.0,3.0
5,3229,I think they should be taken off the shelves i...,2.0,2.0
6,3230,Katherine Paterson made a valid point in stati...,4.0,4.0
7,3231,There are things in the world that are not men...,3.0,4.0
8,3232,What do you go to the library for? In earlier...,3.0,4.0
9,3233,A library card is one thing a kid always looke...,4.0,4.0


In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error
sklearn.metrics.mean_squared_error(df_to_csv['score'],df_to_csv['predicted_score'])

0.37222221

In [ ]:
import skll
skll.metrics.kappa(df_to_csv['score'],df_to_csv['predicted_score'],weights='quadratic')

0.6068075117370892

In [ ]:
import skll
skll.metrics.kappa(df_to_csv['score'],df_to_csv['predicted_score'],weights='quadratic',allow_off_by_one=True)

0.9159908990140598

In [ ]:
df_to_csv.to_csv('/content/drive/MyDrive/capstone_data/bert_2.csv.csv')